In [1]:
import os
import string
import re
import torch
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import numpy as np 
import pandas as pd

from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
from statistics import mean
from sentence_transformers import SentenceTransformer

from sklearn.metrics import roc_auc_score

from tqdm import tqdm
from matplotlib import style
style.use('default')

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = 'true'
os.environ["TOKENIZERS_PARALLELISM"] = "true"


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
FOLDS = 5

In [3]:
STRANSFORMERS = {
    'sentence-transformers/paraphrase-mpnet-base-v2': ('mpnet', 768),
    'sentence-transformers/bert-base-wikipedia-sections-mean-tokens': ('wikipedia', 768)
}

In [4]:
train = pd.read_csv("./train.csv", index_col='id')

In [5]:
def get_encode(df, encoder, name):    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
    model = SentenceTransformer(
        encoder, 
        cache_folder=f'./hf_{name}/'
    )
    model.to(device)
    model.eval()
    return np.array(model.encode(df['excerpt']))

def get_embeddings(df, emb=None, tolist=True):
    
    ret = pd.DataFrame(index=df.index)
    
    for e, s in STRANSFORMERS.items():
        if emb and s[0] not in emb:
            continue
        
        ret[s[0]] = list(get_encode(df, e, s[0]))
        if tolist:
            ret = pd.concat(
                [ret, pd.DataFrame(
                    ret[s[0]].tolist(),
                    columns=[f'{s[0]}_{x}' for x in range(s[1])],
                    index=ret.index)],
                axis=1, copy=False, sort=False)
    
    return ret        

def clean_text(text):
    
    table = text.maketrans(
        dict.fromkeys(string.punctuation))
    
    words = word_tokenize(
        text.lower().strip().translate(table))
    words = [word for word in words if word not in stopwords.words('english')]
    lemmed = [WordNetLemmatizer().lemmatize(word) for word in words]    
    return " ".join(lemmed)

def get_sentence_lengths(text):
    
    tokened = sent_tokenize(text)
    lengths = []
    
    for idx,i in enumerate(tokened):
        splited = list(i.split(" "))
        lengths.append(len(splited))
        
    return (max(lengths),
            min(lengths),
            round(mean(lengths), 3))

def create_features(df):
    
    df_f = pd.DataFrame(index=df.index)
    df_f['text_len'] = df['excerpt'].apply(len)
    df_f['text_clean_len' ]= df['clean_excerpt'].apply(len)
    df_f['text_len_div'] = df_f['text_clean_len' ] / df_f['text_len']
    df_f['text_word_count'] = df['clean_excerpt'].apply(
        lambda x : len(x.split(' ')))
    
    df_f[['max_len_sent','min_len_sent','avg_len_sent']] = \
        df.apply(
            lambda x: get_sentence_lengths(x['excerpt']),
            axis=1, result_type='expand')
    
    return df_f

def getWordsFromURL(url):
    return re.compile(r'[\:/?=\-&.]+',re.UNICODE).split(url)


In [6]:
train['license'] = train['license'].fillna('nan').astype('category').cat.codes
train['clean_excerpt'] = train['excerpt'].apply(clean_text)
train['url_legal'] = train['url_legal'].fillna("nan").apply(getWordsFromURL).apply(
    lambda x: " ".join(x))

train = pd.concat(
    [train, create_features(train)],
    axis=1, copy=False, sort=False)

basic_f_columns = ['text_len', 'text_clean_len', 'text_len_div',
       'text_word_count', 'max_len_sent', 'min_len_sent', 'avg_len_sent']

In [7]:
num_bins = int(np.floor(1 + np.log2(len(train))))
print(f"Bins: {num_bins}")
train['target_q'], bin_edges = pd.qcut(
    train['target'],
    q=num_bins,
    labels=False,
    retbins=True,
    precision=0)

Bins: 12


## Catboost classifier with text and embeddings 

In [8]:
def fit_model_classifier(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=5000,
        eval_metric='AUC',
        od_type='Iter',
        od_wait=500,
        l2_leaf_reg=10,
        bootstrap_type='Bernoulli',
        subsample=0.7,
        thread_count=20,
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=test_pool,
        verbose=100,
        plot=False,
        use_best_model=True)

def get_oof_classifier(
        n_folds, x_train, y, embedding_features,
        cat_features, text_features, tpo, seeds,
        num_bins, emb=None, tolist=True, gpu=True):
    
    ntrain = x_train.shape[0]
        
    oof_train = np.zeros((len(seeds), ntrain, num_bins))    
    models = {}
    for iseed, seed in enumerate(seeds):
        kf = StratifiedKFold(
            n_splits=n_folds,
            shuffle=True,
            random_state=seed)          
        for i, (tr_i, t_i) in enumerate(kf.split(x_train, y)):
            if emb and len(emb) > 0:
                x_tr = pd.concat(
                    [x_train.iloc[tr_i, :],
                     get_embeddings(x_train.iloc[tr_i, :], emb, tolist)],
                    axis=1, copy=False, sort=False)
                x_te = pd.concat(
                    [x_train.iloc[t_i, :],
                     get_embeddings(x_train.iloc[t_i, :], emb, tolist)],
                    axis=1, copy=False, sort=False)
                columns = [x for x in x_tr if (x not in ['excerpt'])]  
                if not embedding_features:
                    for c in emb:
                        columns.remove(c)
            else:
                x_tr = x_train.iloc[tr_i, :]
                x_te = x_train.iloc[t_i, :]
                columns = [x for x in x_tr if (x not in ['excerpt'])] 
            x_tr = x_tr[columns]
            x_te = x_te[columns]                
            y_tr = y[tr_i]            
            y_te = y[t_i]
            train_pool = Pool(
                data=x_tr, label=y_tr, cat_features=cat_features,
                embedding_features=embedding_features, text_features=text_features)
            valid_pool = Pool(
                data=x_te, label=y_te, cat_features=cat_features,
                embedding_features=embedding_features, text_features=text_features)
            task_type='GPU' if gpu else 'CPU'
            model = fit_model_classifier(
                train_pool, valid_pool,
                random_seed=seed,
                task_type=task_type,
                text_processing=tpo
            )
            oof_train[iseed, t_i, :] = model.predict_proba(valid_pool)
            models[(seed, i)] = model
            
    oof_train = oof_train.mean(axis=0)
    
    return oof_train, models

In [9]:
results = {}

In [10]:
tpo = {
    'tokenizers': [
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number']
        }
    ],
    'dictionaries': [
        {
            'dictionary_id': 'Word',
            'token_level_type': 'Word',
            'occurrence_lower_bound': '2'
        },
        {
            'dictionary_id': 'Bigram',
            'token_level_type': 'Word',
            'gram_order': '2',
            'occurrence_lower_bound': '2'
        },
        {
            'dictionary_id': 'Trigram',
            'token_level_type': 'Word',
            'gram_order': '3',
            'occurrence_lower_bound': '2'
        },
    ],
    'feature_processing': {
        '0': [
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Word'],
                'feature_calcers': ['BoW']
            },
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Bigram'],
                'feature_calcers': ['BoW']
            },
        ]
    }
}

tpo_2 = {
    'tokenizers': [
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number']
        }
    ],
    'dictionaries': [
        {
            'dictionary_id': 'Word',
            'token_level_type': 'Word',
            'occurrence_lower_bound': '2'
        },
        {
            'dictionary_id': 'Bigram',
            'token_level_type': 'Word',
            'gram_order': '2',
            'occurrence_lower_bound': '2'
        },
        {
            'dictionary_id': 'Trigram',
            'token_level_type': 'Word',
            'gram_order': '3',
            'occurrence_lower_bound': '2'
        },
    ],
    'feature_processing': {
        '0': [
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Word'],
                'feature_calcers': ['BoW', 'BM25']
            },
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Bigram', 'Trigram'],
                'feature_calcers': ['BoW']
            },
        ],
        '1': [
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Word'],
                'feature_calcers': ['BoW']
            },
            {
                'tokenizers_names': ['Sense'],
                'dictionaries_names': ['Bigram'],
                'feature_calcers': ['BoW']
            },
        ]
    }
}

In [22]:
params = {}

for gpu in [True, False]:
    postfix = '_gpu' if gpu else ""
        
    params.update({
        'basic_f_columns' + postfix: {
            'x_train': train[
                ['license', 'url_legal'] + basic_f_columns],
            'embedding_features': None,
            'text_features': ['url_legal'],
            'tpo': tpo,
            'emb': None,
            'tolist': False,
            'gpu': gpu
        },
        'emb_basic_f_columns' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt'] + basic_f_columns],
            'embedding_features': ['mpnet', 'wikipedia'],
            'text_features': ['url_legal'],
            'tpo': tpo,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': False,
            'gpu': gpu
        },  
        'emb' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt']],
            'embedding_features': ['mpnet', 'wikipedia'],
            'text_features': ['url_legal'],
            'tpo': tpo,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': False,
            'gpu': gpu
        },          
        'emb.tolist()' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt']],
            'embedding_features': None,
            'text_features': ['url_legal'],
            'tpo': tpo,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': True,
            'gpu': gpu
        },
        'emb_emb.tolist()' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt']],
            'embedding_features': ['mpnet', 'wikipedia'],
            'text_features': ['url_legal'],
            'tpo': tpo,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': True,
            'gpu': gpu
        },        
        'emb_text' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt', 'clean_excerpt']],
            'embedding_features': ['mpnet', 'wikipedia'],
            'text_features': ['clean_excerpt', 'url_legal'],
            'tpo': tpo_2,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': False,
            'gpu': gpu
        }, 
        'text' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'clean_excerpt']],
            'embedding_features': None,
            'text_features': ['clean_excerpt', 'url_legal'],
            'tpo': tpo_2,
            'emb': None,
            'tolist': False,
            'gpu': gpu
        },        
        'text_emb.tolist()' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'excerpt', 'clean_excerpt']],
            'embedding_features': None,
            'text_features': ['clean_excerpt', 'url_legal'],
            'tpo': tpo_2,
            'emb': ['mpnet', 'wikipedia'],
            'tolist': True,
            'gpu': gpu
        },     
        'text_basic_f_columns' + postfix: {
            'x_train': train[
                ['license', 'url_legal', 'clean_excerpt'] + basic_f_columns],
            'embedding_features': None,
            'text_features': ['clean_excerpt', 'url_legal'],
            'tpo': tpo_2,
            'emb': None,
            'tolist': False,
            'gpu': gpu
        }              
    })

results = {}

for k,v in params.items():
    results[k] = get_oof_classifier(
        n_folds=FOLDS,  
        y=train['target_q'].values,
        cat_features=['license'],
        seeds=[0, 42, 888],
        num_bins=num_bins,
        **v
    )

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6609356	best: 0.6609356 (0)	total: 65.6ms	remaining: 5m 27s
100:	test: 0.7251144	best: 0.7251144 (100)	total: 5.59s	remaining: 4m 31s
200:	test: 0.7296765	best: 0.7297695 (195)	total: 11s	remaining: 4m 22s
300:	test: 0.7324436	best: 0.7327796 (275)	total: 16.4s	remaining: 4m 16s
400:	test: 0.7336192	best: 0.7338391 (364)	total: 21.7s	remaining: 4m 9s
500:	test: 0.7345147	best: 0.7348569 (479)	total: 26.8s	remaining: 4m
600:	test: 0.7358188	best: 0.7358319 (598)	total: 32.1s	remaining: 3m 54s
700:	test: 0.7361944	best: 0.7363090 (681)	total: 37.3s	remaining: 3m 49s
800:	test: 0.7350876	best: 0.7363090 (681)	total: 42.6s	remaining: 3m 43s
900:	test: 0.7351208	best: 0.7363090 (681)	total: 47.7s	remaining: 3m 37s
1000:	test: 0.7346677	best: 0.7363090 (681)	total: 53.1s	remaining: 3m 32s
1100:	test: 0.7345634	best: 0.7363090 (681)	total: 58.5s	remaining: 3m 27s
bestTest = 0.7363090008
bestIteration = 681
Shrink model to first 682 iterations.


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6678897	best: 0.6678897 (0)	total: 66.6ms	remaining: 5m 32s
100:	test: 0.7393124	best: 0.7393124 (100)	total: 5.36s	remaining: 4m 19s
200:	test: 0.7416965	best: 0.7417861 (196)	total: 10.7s	remaining: 4m 15s
300:	test: 0.7430698	best: 0.7437998 (268)	total: 16.1s	remaining: 4m 11s
400:	test: 0.7445016	best: 0.7445016 (400)	total: 21.6s	remaining: 4m 7s
500:	test: 0.7441989	best: 0.7446846 (416)	total: 27s	remaining: 4m 2s
600:	test: 0.7434309	best: 0.7446846 (416)	total: 32.4s	remaining: 3m 57s
700:	test: 0.7428239	best: 0.7446846 (416)	total: 37.7s	remaining: 3m 51s
800:	test: 0.7427927	best: 0.7446846 (416)	total: 42.9s	remaining: 3m 45s
900:	test: 0.7420183	best: 0.7446846 (416)	total: 48.2s	remaining: 3m 39s
bestTest = 0.7446845604
bestIteration = 416
Shrink model to first 417 iterations.


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


...

0:	test: 0.6998229	best: 0.6998229 (0)	total: 64ms	remaining: 5m 19s
100:	test: 0.7512216	best: 0.7516003 (95)	total: 5.85s	remaining: 4m 43s
200:	test: 0.7550090	best: 0.7550100 (197)	total: 11.9s	remaining: 4m 43s
300:	test: 0.7568607	best: 0.7574844 (267)	total: 17.9s	remaining: 4m 40s
400:	test: 0.7566883	best: 0.7575249 (308)	total: 23.8s	remaining: 4m 33s
500:	test: 0.7570151	best: 0.7575249 (308)	total: 29.7s	remaining: 4m 26s
600:	test: 0.7578743	best: 0.7583119 (549)	total: 35.6s	remaining: 4m 20s
700:	test: 0.7584868	best: 0.7586775 (697)	total: 41.6s	remaining: 4m 15s
800:	test: 0.7587434	best: 0.7591820 (788)	total: 47.7s	remaining: 4m 9s
900:	test: 0.7574131	best: 0.7591820 (788)	total: 53.7s	remaining: 4m 4s
1000:	test: 0.7579183	best: 0.7591820 (788)	total: 59.8s	remaining: 3m 58s
1100:	test: 0.7583123	best: 0.7591820 (788)	total: 1m 5s	remaining: 3m 52s
1200:	test: 0.7589578	best: 0.7591820 (788)	total: 1m 11s	remaining: 3m 47s
bestTest = 0.7591820033
bestIteration = 78

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6786694	best: 0.6786694 (0)	total: 62.5ms	remaining: 5m 12s
100:	test: 0.7497863	best: 0.7498713 (96)	total: 5.21s	remaining: 4m 12s
200:	test: 0.7560692	best: 0.7560692 (200)	total: 11.1s	remaining: 4m 24s
300:	test: 0.7575433	best: 0.7581726 (275)	total: 16.9s	remaining: 4m 24s
400:	test: 0.7585527	best: 0.7595553 (382)	total: 22.9s	remaining: 4m 22s
500:	test: 0.7585893	best: 0.7595553 (382)	total: 28.7s	remaining: 4m 18s
600:	test: 0.7595986	best: 0.7598368 (592)	total: 34.7s	remaining: 4m 14s
700:	test: 0.7571503	best: 0.7598368 (592)	total: 40.7s	remaining: 4m 9s
800:	test: 0.7576939	best: 0.7598368 (592)	total: 46.6s	remaining: 4m 4s
900:	test: 0.7563960	best: 0.7598368 (592)	total: 52.4s	remaining: 3m 58s
1000:	test: 0.7569002	best: 0.7598368 (592)	total: 58.4s	remaining: 3m 53s
bestTest = 0.7598368019
bestIteration = 592
Shrink model to first 593 iterations.


AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6648843	best: 0.6648843 (0)	total: 77.3ms	remaining: 6m 26s
100:	test: 0.7147351	best: 0.7149610 (99)	total: 6.13s	remaining: 4m 57s
200:	test: 0.7211752	best: 0.7212366 (184)	total: 12s	remaining: 4m 46s
300:	test: 0.7236008	best: 0.7236840 (298)	total: 17.9s	remaining: 4m 39s
400:	test: 0.7246715	best: 0.7258125 (368)	total: 23.7s	remaining: 4m 32s
500:	test: 0.7265090	best: 0.7271170 (472)	total: 29.6s	remaining: 4m 25s
600:	test: 0.7276249	best: 0.7276249 (600)	total: 35.5s	remaining: 4m 19s
700:	test: 0.7280965	best: 0.7284487 (656)	total: 41.4s	remaining: 4m 13s
800:	test: 0.7290426	best: 0.7292334 (795)	total: 47.4s	remaining: 4m 8s
900:	test: 0.7288010	best: 0.7296987 (821)	total: 53.2s	remaining: 4m 2s
1000:	test: 0.7280884	best: 0.7296987 (821)	total: 58.8s	remaining: 3m 54s
1100:	test: 0.7282351	best: 0.7296987 (821)	total: 1m 4s	remaining: 3m 49s
1200:	test: 0.7288128	best: 0.7296987 (821)	total: 1m 10s	remaining: 3m 43s
1300:	test: 0.7302227	best: 0.7303127 (129

In [40]:
res = {}
for k,v in results.items():
    if 'gpu' in str(k):
        name = k[0:-4]
        auc_gpu = roc_auc_score(train['target_q'], v[0], multi_class="ovo")
    else:
        auc_cpu = roc_auc_score(train['target_q'], v[0], multi_class="ovo")        
        name = k
    if name in res:
        res[name] = res[name] + [auc_cpu]
    else:
        res[name] = [auc_gpu]
pd.DataFrame.from_dict(res, orient='index', columns=['AUC(GPU)', 'AUC(CPU)'])

,AUC(GPU),AUC(CPU)
basic_f_columns,0.684407,0.685256
emb_basic_f_columns,0.724230,0.674038
emb,0.717203,0.571230
emb.tolist(),0.750460,0.748209
emb_emb.tolist(),0.764183,0.746998
emb_text,0.626926,0.586988
text,0.672553,0.673705
text_emb.tolist(),0.752005,0.747549
text_basic_f_columns,0.698935,0.701273
